In [ ]:
#%pip install shap
#%pip install featurewiz
#import shap
import pickle
import numpy as np
import pandas as pd
import seaborn as sn
#import featurewiz as FW
from sklearn.svm import SVC
from sklearn import metrics
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, space_eval
from sklearn.metrics import roc_auc_score, classification_report, roc_curve, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#from bayes_opt import BayesianOptimization


In [ ]:
data = pd.read_csv('gait3.csv')
data['Subject'].unique()

In [ ]:
g = data.groupby('PAD').mean()

In [ ]:
data = data.groupby("Subject").transform(lambda x: x.fillna(x.mean()))

In [ ]:
def label (row):
  if row['PAD'] == 0:
    return "Healthy"
  if row['PAD'] == 1:
    return "PAD"

data['Condition'] = data.apply(lambda row: label(row), axis=1)
data

In [ ]:
data.rename(columns={'StepTime': 'StepTime', 'StanceTime': 'StanceTime', 'StrideTime': 'StrideTime', 'SwingTime': 'SwingTime',
                     'StepTimeVrl': 'StepTime SV', 'StepTimeV': 'StepTime MV', 'StepTimeA': 'StepTime SA',
                     'StanceTimeVrl': 'StanceTime SV', 'StanceTimeV': 'StanceTime MV', 'StanceTimeA': 'StanceTime SA',
                     'StrideTimeVrl': 'StrideTime SV', 'StrideTimeV': 'StrideTime MV', 'StrideTimeA': 'StrideTime SA',
                     'SwingTimeVrl': 'SwingTime SV', 'SwingTimeV': 'SwingTime MV', 'SwingTimeA': 'SwingTime SA',}, inplace=True)

In [ ]:
d1 = data[['StepTime', 'StanceTime',	'StrideTime',	'SwingTime', 'Condition']]
d1 = pd.melt(d1, id_vars='Condition', value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(1, sharex=False, sharey=False, gridspec_kw={'hspace': 0}, figsize=(15, 5))
bill = sns.load_dataset("tips")
sns.boxplot(x="variable", y="value", hue="Condition", data=d1, palette="Dark2", showfliers = False)
ax.set(xlabel='Gait Characteristic', ylabel='Time (1 / 60 S)')
[ax.axvline(x, color = 'black', linestyle='--') for x in [.5,1.5,2.5]]
ax.tick_params(axis='x', rotation=45)

In [ ]:
d2 = data.drop(['StepTime', 'StanceTime',	'StrideTime',	'SwingTime', 'StrideTime SA' , 'PAD'], axis=1)
d2 = pd.melt(d2, id_vars='Condition', value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)

In [ ]:
fig, ax = plt.subplots(1, sharex=False, sharey=False, gridspec_kw={'hspace': 0}, figsize=(15, 5))
bill = sns.load_dataset("tips")
sns.boxplot(x="variable", y="value", hue="Condition", data=d2, palette="Dark2", showfliers = False)
ax.set(xlabel='Gait Characteristic', ylabel='Value')
[ax.axvline(x, color = 'black', linestyle='--') for x in [.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5]]
ax.tick_params(axis='x', rotation=45)

# Analysis

In [ ]:
data = pd.read_csv('gait3.csv')
data.rename(columns={'StepTime': 'StepTime', 'StanceTime': 'StanceTime', 'StrideTime': 'StrideTime', 'SwingTime': 'SwingTime',
                     'StepTimeVrl': 'StepTime SV', 'StepTimeV': 'StepTime MV', 'StepTimeA': 'StepTime SA',
                     'StanceTimeVrl': 'StanceTime SV', 'StanceTimeV': 'StanceTime MV', 'StanceTimeA': 'StanceTime SA',
                     'StrideTimeVrl': 'StrideTime SV', 'StrideTimeV': 'StrideTime MV', 'StrideTimeA': 'StrideTime SA',
                     'SwingTimeVrl': 'SwingTime SV', 'SwingTimeV': 'SwingTime MV', 'SwingTimeA': 'SwingTime SA',}, inplace=True)
dataset_train = data[data['Subject'].isin(['C16','C17','C19','C20','C21','C22','C23','C25','C31','C33','C34','C35','C36',
                                           'C37','C39','C40','C41', 'C50', 'C57'
                                           'P445','P527','P436','P440','P513','P507','P432','P441','P488','P433','P466'
                                           ,'P484','P483','P482','P469','P462','P444','P460','P487','P528'
                                           ])]

dataset_test = data[data['Subject'].isin(['C11','C38','C43','C48','C49','C53','P459','P458','P457','P450',
                                          'P473','P477','P503'])]

In [ ]:
dataset_train.Subject.isin(dataset_test.Subject).unique()

In [ ]:
dataset_train['Subject'].unique()

In [ ]:
 dataset_train = dataset_train.groupby("Subject").transform(lambda x: x.fillna(x.mean()))
 dataset_test = dataset_test.groupby("Subject").transform(lambda x: x.fillna(x.mean()))

 dataset_train = shuffle(dataset_train, random_state=42)

In [ ]:
dataset_train.columns

In [ ]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from statsmodels.stats.outliers_influence import variance_inflation_factor

numeric_cols = [col for col in dataset_train.columns if is_numeric_dtype(dataset_train[col])]
df_numeric = dataset_train[numeric_cols]
df_numeric = df_numeric.drop(['PAD','StrideTime SV', 'StrideTime', 'SwingTime MV', 'StanceTime SA', 'StanceTime SV', 'StepTime', 'SwingTime SV', 'StepTime MV', 'StepTime SV', 'StanceTime', 'StrideTime MV'],1)
# Calculate the VIF for each independent variable
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_numeric.values, i) for i in range(df_numeric.shape[1])]
vif["features"] = df_numeric.columns
vif_sorted = vif.sort_values(by='VIF Factor', ascending=False)

# Print the VIF values for each independent variable
print(vif_sorted)

In [ ]:
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Load your dataset into a pandas dataframe
X = dataset_train[['PAD']]
# Scale the input variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Calculate the VIF for each independent variable
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_scaled, i) for i in range(X.shape[1])]
vif["features"] = X.columns

# Print the VIF values for each independent variable
print(vif)


In [ ]:
#X = dataset_train.drop(['PAD'], 1)
#y = dataset_train['PAD']

In [ ]:
FW.featurewiz(dataname=dataset_train, target='PAD',feature_egg = 'interaction', corr_limit=0.90, verbose=2)

In [ ]:
X_train = dataset_train[['StepTime', 'StanceTime', 'StrideTime', 'SwingTime',
       'StepTime SV', 'StepTime MV', 'StepTime SA', 'StanceTime SV', 'StanceTime MV',
       'StanceTime SA', 'StrideTime SV', 'StrideTime MV', 'StrideTime SA',
       'SwingTime SV', 'SwingTime MV', 'SwingTime SA']]
y_train = dataset_train[['PAD']]

X_test = dataset_test[['StepTime', 'StanceTime', 'StrideTime', 'SwingTime',
       'StepTime SV', 'StepTime MV', 'StepTime SA', 'StanceTime SV', 'StanceTime MV',
       'StanceTime SA', 'StrideTime SV', 'StrideTime MV', 'StrideTime SA',
       'SwingTime SV', 'SwingTime MV', 'SwingTime SA']]
y_test = dataset_test[['PAD']]

### Logit

In [ ]:
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()
print(result.summary2())

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
# Dropping first 4:
X_train = dataset_train[[
       'StepTimeVrl', 'StepTimeV', 'StepTimeA', 'StanceTimeVrl', 'StanceTimeV',
       'StanceTimeA', 'StrideTimeVrl', 'StrideTimeV', 'StrideTimeA',
       'SwingTimeVrl', 'SwingTimeV', 'SwingTimeA']]

X_test = dataset_test[[
       'StepTimeVrl', 'StepTimeV', 'StepTimeA', 'StanceTimeVrl', 'StanceTimeV',
       'StanceTimeA', 'StrideTimeVrl', 'StrideTimeV', 'StrideTimeA',
       'SwingTimeVrl', 'SwingTimeV', 'SwingTimeA']]

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print(classification_report(y_test, y_pred))
print("Precision:{}".format(precision_score(y_pred,y_test)))
print("Recall:{}".format(recall_score(y_pred,y_test)))
print("F1 Score:{}".format((f1_score(y_pred,y_test))))
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, range(2), range(2))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
plt.title("Accuracy: %.2f%%" % (accuracy_score(y_test, y_pred)*100))
plt.xlabel("Predictions")
plt.ylabel("Actual")
plt.show()

In [ ]:
data = pd.read_csv('gait3.csv')
dataset_test = data[data['Subject'].isin(['C11','C38','C43','C48','C49','C53','P459','P458','P457','P450',
                                          'P473','P477','P503'])]
sub = pd.DataFrame(dataset_test['Subject'])
sub['actual'] = y_test
sub['prediction'] = y_pred
f = sub.groupby("Subject", as_index = False).mean()

print(classification_report(f.actual, f.prediction))
print("Precision:{}".format(precision_score(f.prediction,f.actual)))
print("Recall:{}".format(recall_score(f.prediction,f.actual)))
print("F1 Score:{}".format((f1_score(f.prediction,f.actual))))

cm = confusion_matrix(f.actual, f.prediction)
df_cm = pd.DataFrame(cm, range(2), range(2))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
plt.title("Accuracy: %.2f%%" % (accuracy_score(f.actual, f.prediction)*100))
plt.xlabel("Predictions")
plt.ylabel("Actual")
plt.show()

In [ ]:
f = f[['Subject', 'prediction', 'actual']]
f.columns = ['Subject', 'Prediction', 'Actual']
f

In [ ]:
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

Final edit

In [ ]:
# Applying VIF test
X_train = dataset_train[['StepTime SA', 'SwingTime SA', 'SwingTime', 'StrideTime SA', 'StanceTime MV']]
X_test = dataset_test[['StepTime SA', 'SwingTime SA', 'SwingTime', 'StrideTime SA', 'StanceTime MV']]

In [ ]:
X_train = dataset_train[['StanceTime SV', 'SwingTime SV', 'StrideTime SV', 'StepTime MV', 'StepTime SA', 'SwingTime SA',
                         'StanceTime SA', 'StrideTime SA']]

X_test = dataset_test[['StanceTime SV', 'SwingTime SV', 'StrideTime SV', 'StepTime MV', 'StepTime SA', 'SwingTime SA',
                         'StanceTime SA', 'StrideTime SA']]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score

# Assume X_train, y_train, X_test, and y_test are your training and test data
logistic_regression = LogisticRegression()
accuracy_scores = cross_val_score(logistic_regression, X_train, y_train, cv=5)

print(f"Cross-validation accuracy: {accuracy_scores.mean():.2f} +/- {accuracy_scores.std():.2f}")

In [ ]:
logistic_regression.fit(X_train, y_train)
y_pred = logistic_regression.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Test set accuracy: {accuracy:.2f}")

In [ ]:
data = pd.read_csv('gait3.csv')
dataset_test = data[data['Subject'].isin(['C11','C38','C43','C48','C49','C53','P459','P458','P457','P450',
                                          'P473','P477','P503'])]
sub = pd.DataFrame(dataset_test['Subject'])
sub['actual'] = y_test
sub['prediction'] = y_pred
f = sub.groupby("Subject", as_index = False).mean()
f['prediction'] = np.round(f['prediction'])

print(classification_report(f.actual, f.prediction))
print("Precision:{}".format(precision_score(f.prediction,f.actual)))
print("Recall:{}".format(recall_score(f.prediction,f.actual)))
print("F1 Score:{}".format((f1_score(f.prediction,f.actual))))

cm = confusion_matrix(f.actual, f.prediction)
df_cm = pd.DataFrame(cm, range(2), range(2))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
plt.title("Accuracy: %.2f%%" % (accuracy_score(f.actual, f.prediction)*100))
plt.xlabel("Predictions")
plt.ylabel("Actual")
plt.show()

In [ ]:
# Assume logistic_regression is your fitted logistic regression model
feature_names = ['StepTime', 'StanceTime', 'StrideTime', 'SwingTime', 'StepTimeVrl',
       'StepTimeV', 'StepTimeA', 'StanceTimeVrl', 'StanceTimeV', 'StanceTimeA',
       'StrideTimeVrl', 'StrideTimeV', 'StrideTimeA', 'SwingTimeVrl',
       'SwingTimeV', 'SwingTimeA']
coef_abs = np.abs(logistic_regression.coef_[0])
sorted_idx = np.argsort(coef_abs)[::-1]

# Print the feature names and corresponding coefficients in descending order of importance
for i in sorted_idx:
    print(f"{feature_names[i]}: {logistic_regression.coef_[0][i]:.4f}")

### Random Forest

In [ ]:
data = pd.read_csv('gait3.csv')
data.rename(columns={'StepTime': 'StepTime', 'StanceTime': 'StanceTime', 'StrideTime': 'StrideTime', 'SwingTime': 'SwingTime',
                     'StepTimeVrl': 'StepTime SV', 'StepTimeV': 'StepTime MV', 'StepTimeA': 'StepTime SA',
                     'StanceTimeVrl': 'StanceTime SV', 'StanceTimeV': 'StanceTime MV', 'StanceTimeA': 'StanceTime SA',
                     'StrideTimeVrl': 'StrideTime SV', 'StrideTimeV': 'StrideTime MV', 'StrideTimeA': 'StrideTime SA',
                     'SwingTimeVrl': 'SwingTime SV', 'SwingTimeV': 'SwingTime MV', 'SwingTimeA': 'SwingTime SA',}, inplace=True)
dataset_train = data[data['Subject'].isin(['C16','C17','C19','C20','C21','C22','C23','C25','C31','C33','C34','C35','C36',
                                           'C37','C39','C40','C41', 'C50', 'C57'
                                           'P445','P527','P436','P440','P513','P507','P432','P441','P488','P433','P466'
                                           ,'P484','P483','P482','P469','P462','P444','P460','P487','P528'
                                           ])]

dataset_test = data[data['Subject'].isin(['C11','C38','C43','C48','C49','C53','P459','P458','P457','P450',
                                          'P473','P477','P503'])]

In [ ]:
 dataset_train = dataset_train.groupby("Subject").transform(lambda x: x.fillna(x.mean()))
 dataset_test = dataset_test.groupby("Subject").transform(lambda x: x.fillna(x.mean()))

 dataset_train = shuffle(dataset_train, random_state=42)

In [ ]:
X = dataset_train.drop(['PAD'],1)
Xt = dataset_test.drop(['PAD'],1)
y_train = dataset_train[['PAD']]
y_test = dataset_test[['PAD']]
y = y_train.copy()
yt = y_test.copy()

In [ ]:
X = dataset_train[['SwingTime SA', 'SwingTime MV', 'StrideTime MV', 'StepTime SA']]

Xt = dataset_test[['SwingTime SA', 'SwingTime MV', 'StrideTime MV', 'StepTime SA']]

y_train = dataset_train[['PAD']]
y_test = dataset_test[['PAD']]
y = y_train.copy()
yt = y_test.copy()

In [ ]:
from hyperopt import fmin, tpe, hp
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

def hyperopt_train_test(params):
    clf = RandomForestClassifier(**params)
    return cross_val_score(clf, X, y).mean()

def f(params):
    acc = hyperopt_train_test(params)
    return {'loss': -acc, 'status': 'ok'}

space = {
    'max_depth': hp.choice('max_depth', range(1, 20)),
    'max_features': hp.choice('max_features', range(1, len(X.columns))),
    'n_estimators': 70, #hp.choice('n_estimators', range(0, 100)),
    'criterion': hp.choice('criterion', ["gini", "entropy"]),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
    'min_samples_split' : hp.uniform ('min_samples_split', 0, 1)
}

best = fmin(f, space, algo=tpe.suggest, max_evals=100)
print(best)

In [ ]:
best_params = space_eval(space, best)
clf = RandomForestClassifier(**best_params)
clf.fit(X, y)

# Evaluate the classifier on the test set
y_pred = clf.predict(Xt)
accuracy = accuracy_score(yt, y_pred)
print(f'Test accuracy: {accuracy:.3f}')

In [ ]:
data = pd.read_csv('gait3.csv')
dataset_test = data[data['Subject'].isin(['C11','C38','C43','C48','C49','C53','P459','P458','P457','P450',
                                          'P473','P477','P503'])]
sub = pd.DataFrame(dataset_test['Subject'])
sub['actual'] = y_test
sub['prediction'] = y_pred
f = sub.groupby("Subject", as_index = False).mean()

print(classification_report(f.actual, f.prediction))
print("Precision:{}".format(precision_score(f.prediction,f.actual)))
print("Recall:{}".format(recall_score(f.prediction,f.actual)))
print("F1 Score:{}".format((f1_score(f.prediction,f.actual))))

cm = confusion_matrix(f.actual, f.prediction)
df_cm = pd.DataFrame(cm, range(2), range(2))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
plt.title("Accuracy: %.2f%%" % (accuracy_score(f.actual, f.prediction)*100))
plt.xlabel("Predictions")
plt.ylabel("Actual")
plt.show()

In [ ]:
import shap

# Create an explainer object
#explainer = shap.Explainer(clf, X)

# Generate SHAP values for the test set
#shap_values = explainer(Xt)

shap_values = shap.TreeExplainer(clf).shap_values(X)

# Plot the SHAP summary plot
shap.summary_plot(shap_values, Xt, plot_type='bar', feature_names=Xt.columns, show=False)

# Show the plot
plt.show()

In [ ]:
space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.choice('max_depth', range(0,20)),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', range(0,20))
    }

def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'],
                                 )

    accuracy = cross_val_score(model, X_train, y_train, cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 5,
            trials= trials)
best

In [ ]:
X_train = dataset_train[['SwingTimeVrl', 'SwingTimeV', 'StrideTimeVrl', 'StepTimeA']]

X_test = dataset_test[['SwingTimeVrl', 'SwingTimeV', 'StrideTimeVrl', 'StepTimeA']]

In [ ]:
model = RandomForestClassifier(criterion = 'gini',
                                       max_depth = best['max_depth'],
                                       max_features = None,
                                       min_samples_leaf = best['min_samples_leaf'],
                                       min_samples_split = best['min_samples_split'],
                                       n_estimators = best['n_estimators']
                                       )
trainedforest = model.fit(X_train, y_train)
y_pred = trainedforest.predict(X_test)
#print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
with open('RF_model.pickle', 'wb') as handle:
    pickle.dump(trainedforest, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('RF_model.pickle', 'rb') as f:
    trainedforest = pickle.load(f)

y_pred = trainedforest.predict(X_test)
#print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
shap_values = shap.TreeExplainer(model).shap_values(X_train)
shap.summary_plot(shap_values, X_train, feature_names=X_train.columns, plot_type='bar')

# FW: 'StanceTimeV','StepTimeA','StanceTimeA','StepTimeV','StrideTimeA','StrideTime'

In [ ]:
data = pd.read_csv('gait3.csv')
dataset_test = data[data['Subject'].isin(['C11','C38','C43','C48','C49','C53','P459','P458','P457','P450',
                                          'P473','P477','P503'])]
sub = pd.DataFrame(dataset_test['Subject'])
sub['actual'] = y_test
sub['prediction'] = y_pred
f = sub.groupby("Subject", as_index = False).mean()

print(classification_report(f.actual, f.prediction))
print("Precision:{}".format(precision_score(f.prediction,f.actual)))
print("Recall:{}".format(recall_score(f.prediction,f.actual)))
print("F1 Score:{}".format((f1_score(f.prediction,f.actual))))

cm = confusion_matrix(f.actual, f.prediction)
df_cm = pd.DataFrame(cm, range(2), range(2))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
plt.title("Accuracy: %.2f%%" % (accuracy_score(f.actual, f.prediction)*100))
plt.xlabel("Predictions")
plt.ylabel("Actual")
plt.show()

In [ ]:
f = f[['Subject', 'prediction', 'actual']]
f.columns = ['Subject', 'Prediction', 'Actual']
f

### SVM
Drop 4 features

In [ ]:
space = {
      'C': hp.choice('C', np.arange(1,100,1)),
      'kernel': hp.choice('kernel',['linear', 'poly', 'rbf', 'sigmoid']),
      'degree':hp.choice('degree',[1,2,3,4,5]),
      'probability':hp.choice('probability',[True])
      }


def objective(space):
    model = SVC(**space)
    accuracy = cross_val_score(model, X_train, y_train, cv = 5).mean()
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best = fmin(fn=objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 30,
            trials = trials)
best

In [ ]:
model = SVC(C = 35, #np.arange(0,100,1)[best['C']],
            kernel = 'rbf',
            degree = 1,
            probability = 0
            )

trainedsvc = model.fit(X_train, y_train)
y_pred = trainedsvc.predict(X_test)
#print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
data = pd.read_csv('gait3.csv')
dataset_test = data[data['Subject'].isin(['C11','C38','C43','C48','C49','C53','P459','P458','P457','P450',
                                          'P473','P477','P503'])]
sub = pd.DataFrame(dataset_test['Subject'])
sub['actual'] = y_test
sub['prediction'] = y_pred
f = sub.groupby("Subject", as_index = False).mean()

print(classification_report(f.actual, f.prediction))
print("Precision:{}".format(precision_score(f.prediction,f.actual)))
print("Recall:{}".format(recall_score(f.prediction,f.actual)))
print("F1 Score:{}".format((f1_score(f.prediction,f.actual))))

cm = confusion_matrix(f.actual, f.prediction)
df_cm = pd.DataFrame(cm, range(2), range(2))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
plt.title("Accuracy: %.2f%%" % (accuracy_score(f.actual, f.prediction)*100))
plt.xlabel("Predictions")
plt.ylabel("Actual")
plt.show()

In [ ]:
f = f[['Subject', 'prediction', 'actual']]
f.columns = ['Subject', 'Prediction', 'Actual']

In [ ]:
f

### Neural Network

In [ ]:
data = pd.read_csv('gait3.csv')
data.rename(columns={'StepTime': 'StepTime', 'StanceTime': 'StanceTime', 'StrideTime': 'StrideTime', 'SwingTime': 'SwingTime',
                     'StepTimeVrl': 'StepTime SV', 'StepTimeV': 'StepTime MV', 'StepTimeA': 'StepTime SA',
                     'StanceTimeVrl': 'StanceTime SV', 'StanceTimeV': 'StanceTime MV', 'StanceTimeA': 'StanceTime SA',
                     'StrideTimeVrl': 'StrideTime SV', 'StrideTimeV': 'StrideTime MV', 'StrideTimeA': 'StrideTime SA',
                     'SwingTimeVrl': 'SwingTime SV', 'SwingTimeV': 'SwingTime MV', 'SwingTimeA': 'SwingTime SA',}, inplace=True)
dataset_train = data[data['Subject'].isin(['C16','C17','C19','C20','C21','C22','C23','C25','C31','C33','C34','C35','C36',
                                           'C37','C39','C40','C41', 'C50', 'C57'
                                           'P445','P527','P436','P440','P513','P507','P432','P441','P488','P433','P466'
                                           ,'P484','P483','P482','P469','P462','P444','P460','P487','P528'
                                           ])]

dataset_test = data[data['Subject'].isin(['C11','C38','C43','C48','C49','C53','P459','P458','P457','P450',
                                          'P473','P477','P503'])]

In [ ]:
 dataset_train = dataset_train.groupby("Subject").transform(lambda x: x.fillna(x.mean()))
 dataset_test = dataset_test.groupby("Subject").transform(lambda x: x.fillna(x.mean()))

 dataset_train = shuffle(dataset_train, random_state=42)

In [ ]:
X_train = dataset_train.drop(['PAD'],1)
X_test = dataset_test.drop(['PAD'],1)
y_train = dataset_train[['PAD']]
y_test = dataset_test[['PAD']]

In [ ]:
from sklearn.preprocessing import StandardScaler
# Create scaler objects
standard_scaler = StandardScaler()

# Fit and transform the training data using each scaler
X_train_standard = standard_scaler.fit_transform(X_train)

# Transform the test data using each scaler (do NOT fit again!)
X_test_standard = standard_scaler.transform(X_test)

In [ ]:
from hyperopt import fmin, tpe, hp
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, RMSprop, SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

# Define search space for hyperparameters
search_space = {
    'num_hidden_layers': hp.choice('num_hidden_layers', [1, 2, 3]),
    'num_hidden_units': hp.choice('num_hidden_units', [32, 64, 128]),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5),
    'batch_size': hp.choice('batch_size', [16, 32, 64]),
    'activation': hp.choice('activation', ['relu', 'tanh', 'sigmoid']),
    'learning_rate': hp.loguniform('learning_rate', -6, -2),
    'optimizer': hp.choice('optimizer', ['Adam', 'RMSprop', 'SGD']),
    'num_epochs': hp.choice('num_epochs', [10, 20, 30])
}

# Define function to create Keras model
def create_model(params):
    num_hidden_layers = params['num_hidden_layers']
    num_hidden_units = params['num_hidden_units']
    dropout_rate = params['dropout_rate']
    activation = params['activation']
    learning_rate = params['learning_rate']
    optimizer = params['optimizer']

    model = Sequential()
    model.add(Dense(num_hidden_units, activation=activation, input_dim=X_train_standard.shape[1]))
    for i in range(num_hidden_layers):
        model.add(Dense(num_hidden_units, activation=activation))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    if optimizer == 'Adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'SGD':
        opt = SGD(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

# Create Keras classifier object
def keras_wrapper(params):
    return KerasClassifier(build_fn=create_model, verbose=0, epochs=params['num_epochs'], batch_size=params['batch_size'])

# Define objective function for hyperparameter tuning with 5-fold cross-validation
def objective(params):
    # Set parameters for Keras model and compile
    model = create_model(params)

    # Evaluate model with 5-fold cross-validation
    scores = cross_val_score(keras_wrapper(params), X_train_standard, y_train, cv=5, scoring='accuracy')
    mean_score = scores.mean()

    return {'loss': -mean_score, 'status': 'ok'}

# Perform hyperparameter tuning using Tree-structured Parzen Estimator (TPE) algorithm
best_params = fmin(fn=objective, space=search_space, algo=tpe.suggest, max_evals=50)

print(best_params)


In [ ]:
from hyperopt import fmin, tpe, hp
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, RMSprop, SGD  # Add import statement for RMSprop
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

# Define search space for hyperparameters
search_space = {
    'num_hidden_layers': hp.choice('num_hidden_layers', [1, 2, 3]),
    'num_hidden_units': hp.choice('num_hidden_units', [32, 64, 128]),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5),
    'batch_size': hp.choice('batch_size', [16, 32, 64]),
    'activation': hp.choice('activation', ['relu', 'tanh', 'sigmoid']),
    'learning_rate': hp.loguniform('learning_rate', -6, -2),
    'optimizer': hp.choice('optimizer', ['Adam', 'RMSprop', 'SGD'])
}

# Define function to create Keras model
def create_model(params):
  print(params)
  num_hidden_layers = params['num_hidden_layers']
  num_hidden_units = params['num_hidden_units']
  dropout_rate = params['dropout_rate']
  activation = params['activation']
  learning_rate = params['learning_rate']
  optimizer = params['optimizer']

  model = Sequential()
  model.add(Dense(num_hidden_units, activation=activation, input_dim=X_train_standard.shape[1]))
  for i in range(num_hidden_layers):
      model.add(Dense(num_hidden_units, activation=activation))
      model.add(Dropout(dropout_rate))
  model.add(Dense(1, activation='sigmoid'))

    # Compile model
  if optimizer == 'Adam':
      opt = Adam(learning_rate=learning_rate)
  elif optimizer == 'RMSprop':
      opt = RMSprop(learning_rate=learning_rate)
  elif optimizer == 'SGD':
      opt = SGD(learning_rate=learning_rate)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

  return model

# Create Keras classifier object
classifier = KerasClassifier(build_fn=create_model, verbose=0)

# Define objective function for hyperparameter tuning with 5-fold cross-validation
def objective(params):
    # Set parameters for cross-validation
    batch_size = params['batch_size']

    # Set parameters for Keras model and compile
    model = create_model(params)

    # Evaluate model with 5-fold cross-validation
    scores = cross_val_score(classifier, X_train_standard, y_train, cv=5, scoring='accuracy', fit_params={'batch_size': batch_size})
    mean_score = scores.mean()

    return {'loss': -mean_score, 'status': 'ok'}

# Perform hyperparameter tuning using Tree-structured Parzen Estimator (TPE) algorithm
best_params = fmin(fn=objective, space=search_space, algo=tpe.suggest, max_evals=50)

print(best_params)

In [ ]:
def create_model(layers, activation):
    model = Sequential()

    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes, input_dim = X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))

    model.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))

    model.compile(optimizer ='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, BatchNormalization
from keras.layers import LeakyReLU, PReLU, ELU
from keras.layers import Dropout
from keras.activations import relu, sigmoid

In [ ]:
model = KerasClassifier(build_fn = create_model, verbose = 0)

In [ ]:
# layers = [[20], [40,20], [45,30,15]]
# In case your have runtime error, you may need to change the list of lists that specify the layers to a list of tuples
# RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fb8700b6f90>, as the constructor either does not set or modifies parameter layers
# https://stackoverflow.com/questions/59818584/cannot-clone-object-keras-wrappers

layers = [(20,), (5,5,5,5), (10,10,10,10)]

# it indicates that we need a layers as 1 hidden layer with 20 neurons.
# [40,20] will indicates 2 hidden layer, with 40 neurns at first layer, and 20 neurons on 2nd hidden layer
# and so on

# Dont forget to add (,) in (20,) otherwise it will throw an error like : TypeError: 'int' object is not iterable This is because single tuple without comma(,) is treated as int.

In [ ]:
activations = ['sigmoid', 'relu', 'tanh']

param_grid =dict(layers = layers, activation = activations, batch_size = [128, 256], epochs = [30])

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 5)

In [ ]:
grid_result = grid.fit(x_train_sc, y_train_sc)

In [ ]:
grid_result.best_score_, grid_result.best_params_

In [ ]:
y_pred = grid.predict(x_test_sc)
y_pred = (y_pred > 0.5)

In [ ]:
data = pd.read_csv('gait3.csv')
dataset_test = data[data['Subject'].isin(['C11','C38','C43','C48','C49','C53','P459','P458','P457','P450',
                                          'P473','P477','P503'])]
sub = pd.DataFrame(dataset_test['Subject'])
sub['actual'] = y_test_sc
sub['prediction'] = y_pred
f = sub.groupby("Subject", as_index = False).mean()

print(classification_report(f.actual, f.prediction))
print("Precision:{}".format(precision_score(f.prediction,f.actual)))
print("Recall:{}".format(recall_score(f.prediction,f.actual)))
print("F1 Score:{}".format((f1_score(f.prediction,f.actual))))

cm = confusion_matrix(f.actual, f.prediction)
df_cm = pd.DataFrame(cm, range(2), range(2))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
plt.title("Accuracy: %.2f%%" % (accuracy_score(f.actual, f.prediction)*100))
plt.xlabel("Predictions")
plt.ylabel("Actual")
plt.show()

In [ ]:
f = f[['Subject', 'prediction', 'actual']]
f.columns = ['Subject', 'Prediction', 'Actual']
f